In [40]:
from dotenv import load_dotenv
import pinecone
import os
import numpy as np
import pandas as pd

In [8]:
load_dotenv("../../services/inference_service/.env")

True

In [12]:
pinecone.init(
    api_key=os.getenv("PINECONE_API_KEY"),
    environment=os.getenv("PINECONE_ENVIRONMENT"),
)

general_index = pinecone.Index("judieai")


In [78]:
for i in general_index.describe_index_stats()['namespaces'].items():
    print(i)
print(len(general_index.describe_index_stats()['namespaces']))

('CosmoAstro', {'vector_count': 2571})
('APEngLang', {'vector_count': 2348})
('LSAT', {'vector_count': 69120})
('HSChem', {'vector_count': 73274})
('PreAlg', {'vector_count': 6588})
('SocStud', {'vector_count': 2901})
('Psych', {'vector_count': 5295})
('WorldRelig', {'vector_count': 4418})
('APPhys1', {'vector_count': 1953})
('HSStats', {'vector_count': 1985})
('HSPhys', {'vector_count': 553})
('ACTPrep', {'vector_count': 3174})
('APPhys2', {'vector_count': 4734})
('Algebra2', {'vector_count': 3596})
('APEngLit', {'vector_count': 2245})
('WorldHist', {'vector_count': 6061})
('Socio', {'vector_count': 2486})
('MolBio', {'vector_count': 5160})
('APPsych', {'vector_count': 5295})
('APChem', {'vector_count': 73274})
('MultiCalc', {'vector_count': 4256})
('APUSHist', {'vector_count': 2600})
('MSEarthSpace', {'vector_count': 204})
('AncientHist', {'vector_count': 1235})
('OrgChem', {'vector_count': 14718})
('CollAdm', {'vector_count': 820})
('DATPrep', {'vector_count': 14589})
('USHist', {'v

In [86]:
def get_all_vectors(top_k=10000):
    agg_df = pd.DataFrame()
    for item in general_index.describe_index_stats()['namespaces'].items():
        namespace = item[0]
        count_vectors = item[1]['vector_count']

        ids = set()
        df = pd.DataFrame()
        while len(ids) < count_vectors:
            input_vector = np.random.rand(1536).tolist()
            try:
                results = general_index.query(
                    input_vector,
                    top_k=top_k,
                    include_metadata=True,
                    namespace=namespace,
                )
            except:
                continue

            for result in results['matches']:
                if result.id not in ids:
                    ids.add(result.id)
                    row_dict = {"id":result.id,'sentence':result.metadata['Sentence'],'namespace':namespace}
                    df = pd.concat([df, pd.DataFrame([row_dict])], ignore_index=True)
        agg_df = pd.concat([agg_df, df], ignore_index=True)

    return agg_df

In [87]:
input_vector = np.random.rand(1536).tolist()
#print(input_vector)
total_df = get_all_vectors()
print(total_df)

          id                                           sentence namespace
0        394  Some of the most important inventions were inf...   SocStud
1        387  However, it was the railroad industry that aff...   SocStud
2       1755  Significantly, integration helped to break dow...   SocStud
3       1987  Unions can also drive up wages. A union is an ...   SocStud
4       2143  Congress may do what is necessary to the end f...   SocStud
...      ...                                                ...       ...
521546   148  So what do you think would have happened to th...     MSBio
521547   270  Traits are specific, observable characteristic...     MSBio
521548   100  What are traits? Well, traits are an organism'...     MSBio
521549   271  Or they can be health-related, such as a dog's...     MSBio
521550   106  Because of work done by scientists, we now kno...     MSBio

[521551 rows x 3 columns]


In [90]:
total_df.to_csv('/Users/brody/work/data/pinecone_backup.csvt')